# - Capstone project Phase 1 by Timur Taumurzaev -

### 1. Download and Explore Dataset.

####   At first, we need to scrape the data from Wikipedia page and to transform it to a pandas dataframe. To do that we will use BeautifulSoup package and some additional libraries. Let us start with downloading the following libraries and so on.

In [4]:
!pip install beautifulsoup4
!pip install lxml

In [5]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

#### Now, we are ready to scrape our data from Wikipedia as csv file.

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

csv_file = open('Scrape_File.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode','Borough','Neighborhood'])

p = True
b = False
n = False

for td in soup.table.find_all('td'):
    if p:
        post = td.text
        p = False
        b = True
        continue 
    elif b:
        boro = td.text
        b = False
        n = True
        continue 
    elif n:
        neig = td.text
        n = False
        p = True
        csv_writer.writerow([post,boro,neig])
        continue 

csv_file.close()

#### The next step is to clean and transform our data according to the requirements.

In [7]:
TorontoDF = pd.read_csv('Scrape_File.csv')
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [8]:
TorontoDF['Neighborhood'] = TorontoDF['Neighborhood'].str[:-1]
TorontoDF["Borough"].replace('Not assigned', np.nan, inplace=True)
TorontoDF = TorontoDF.dropna()
TorontoDF.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [9]:
TransformDF = TorontoDF
TransformDF1 = TransformDF.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
TransformDF1 = TransformDF1.reset_index()
TransformDF1 = TransformDF1.set_index('PostalCode')
TransformDF1

,Neighborhood
PostalCode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae
...,...
M9N,Weston
M9P,Westmount
M9R,"Kingsview Village, Martin Grove Gardens, Richv..."


In [10]:
TransformDF = TransformDF.drop(['Neighborhood'], axis=1)
TransformDF = TransformDF.drop_duplicates(['PostalCode'])
TransformDF = TransformDF.set_index('PostalCode')
TransformDF

,Borough
PostalCode,
M3A,North York
M4A,North York
M5A,Downtown Toronto
M6A,North York
M7A,Downtown Toronto
...,...
M8X,Etobicoke
M4Y,Downtown Toronto
M7Y,East Toronto


In [12]:
TorontoDF = TransformDF.join(TransformDF1)
TorontoDF.head(10)

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Queen's Park,Not assigned
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


In [14]:
for i in range(TorontoDF.shape[0]):
    if TorontoDF['Neighborhood'][i] == "Not assigned":
        TorontoDF['Neighborhood'][i] = TorontoDF['Borough'][i]
TorontoDF.head(10)

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Queen's Park,Queen's Park
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


In [15]:
TorontoDF.shape

(103, 2)